<h1>Importes necesarios</h1>

In [2]:
'''!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install joblib
!pip install tensorflow
!pip install keras'''

'!pip install scikit-learn\n!pip install pandas\n!pip install numpy\n!pip install joblib\n!pip install tensorflow\n!pip install keras'

In [3]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, precision_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
import joblib
import os
import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input
import warnings

<h1>Pipeline de preparacion y limpieza de datos</h1>

In [5]:
class Limpiar(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.map(lambda x: x.strip().strip("'") if isinstance(x, str) else x).iloc[:, 1:]

class CodificarCategoricos(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()
        self.dummy_columns = None

    def fit(self, X, y=None):
        self.dummy_columns = pd.get_dummies(X.iloc[:, 1:4]).columns
        cols = list(X.columns[1:4])
        if X.shape[1] > 41:
            cols.append(X.columns[41])
        self.scaler.fit(X.drop(columns=cols))
        return self

    def transform(self, X):
        dummies = pd.get_dummies(X.iloc[:, 1:4])
        dummies = dummies.reindex(columns=self.dummy_columns, fill_value=False)
        cols = list(X.columns[1:4])
        if X.shape[1] > 41:
            y = X.iloc[:, 41].map({'Normal': 0, 'Ataque': 1})
            cols.append(X.columns[41])
            X = X.drop(columns=cols)
            X_float = self.scaler.transform(X)
            X_float = pd.DataFrame(X_float, columns=X.columns, index=X.index)
            return pd.concat([X_float, dummies, y], axis=1)
        X = X.drop(columns=cols)
        X_float = self.scaler.transform(X)
        X_float = pd.DataFrame(X_float, columns=X.columns, index=X.index)
        return pd.concat([X_float, dummies], axis=1)

class ImputarNulos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.fillna(X.mean())

pipeline = Pipeline([
    ('limpiar', Limpiar()),
    ('codificar_categoricos', CodificarCategoricos()),
    ('imputar_nulos', ImputarNulos()),
])
joblib.dump(pipeline, "pipeline.pkl")

['pipeline.pkl']

<h1>Ejecutar el pipeline en el set de datos</h1>

In [7]:
inicio = time.time()
def cargar_datos(path):
    return pd.read_csv(path, header=None)

df = cargar_datos("muestra_ataques.csv")
df = pipeline.fit_transform(df)
joblib.dump(pipeline, "pipeline.pkl")
datos = df.to_numpy()

y = datos[:,-1]
X = datos[:,:-1]

X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.05 minutos


<h1>Definir set de entrenamiento del 80% y set de prueba del 20%</h1>

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
modelos=[]

<h1>Entrenamiento de modelo de regresión logistica</h1>

In [11]:
inicio = time.time()
lr = LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    max_iter=1000,
    class_weight='balanced',
    random_state=123
)
model = lr.fit(X_train, y_train)
modelos.append([model, 'lr'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.01 minutos


<h1>Busqueda aleatoria de hiperparametros para arbol de clasificación con validación cruzada de 4 carpetas usando concurrencia</h1>

In [13]:
inicio = time.time()
clf = DecisionTreeClassifier(random_state=1234)
param_grid = {'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,None],
              'criterion': ['gini','entropy'],
              'min_samples_split': [2,3,4,5,6,7,8],
              'min_samples_leaf': [1,2,3,4,5,6],
              'ccp_alpha': [0,0.0001,0.001,0.01,0.1,1]
             }
random_search = RandomizedSearchCV(clf, param_grid, cv=4, random_state=123, n_jobs=-1, n_iter=500)
random_search.fit(X_train, y_train)
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  1.17 minutos


<h1>Entrenamiento de modelo de arbol de decision con mejores hiperparametros</h1>

In [15]:
inicio = time.time()
clf = DecisionTreeClassifier(max_depth = random_search.best_params_['max_depth'],
                             criterion = random_search.best_params_['criterion'],
                             min_samples_split = random_search.best_params_['min_samples_split'],
                             min_samples_leaf = random_search.best_params_['min_samples_leaf'],
                             ccp_alpha =random_search.best_params_['ccp_alpha'])
model = clf.fit(X_train, y_train)
modelos.append([model, 'clf'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.01 minutos


<h1>Busqueda aleatoria de hiperparametros para modelo K vecinos cercanos con validación cruzada de 4 carpetas sobre una parte del set de entrenamiento usando concurrencia</h1>

In [17]:
inicio = time.time()
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': range(1,60),
              'p': [a/10 for a in range(10,30,5)],
              'weights': ['distance', 'uniform'],
             }
random_search = RandomizedSearchCV(knn, param_grid, cv=4, random_state=123, n_jobs=-1)
random_search.fit(X_train[:round(len(y_train)/4)], y_train[:round(len(y_train)/4)])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  1.09 minutos


<h1>Entrenamiento para modelo de K vecinos cercanos con los mejores hiperparametros</h1>

In [19]:
inicio = time.time()
knn = KNeighborsClassifier(n_neighbors=random_search.best_params_['n_neighbors'],
              p=random_search.best_params_['p'],
              weights=random_search.best_params_['weights'])

model = knn.fit(X_train, y_train)
modelos.append([model, 'knn'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.00 minutos


<h1>Busqueda aleatoria de hiperparametros para máquina de soporte vectorial con validación cruzada de 3 carpetas sobre parte del set de entrenamiento usando concurrencia</h1>

In [21]:
inicio = time.time()
param_grid = {'coef0': [75,100,125],
              'degree': [2,3,4],
              'kernel': ['linear','poly','rbf']
             }
svc = svm.SVC()
grid_search = GridSearchCV(svc, param_grid, cv=4, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.85 minutos


<h1>Entrenamiento para modelo de máquina de soporte vectorial con mejores hiperparametros sobre una parte del set de entrenamiento</h1>

In [23]:
inicio = time.time()
svc = svm.SVC(coef0=grid_search.best_params_['coef0'],
              degree=grid_search.best_params_['degree'],
              kernel=grid_search.best_params_['kernel'])

svc.fit(X_train, y_train)
modelos.append([svc, 'svm'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.07 minutos


<h1>Entrenamiento para modelo de redes neuronales secuenciales con parte del set de entrenamiento, con una capa de entrada, una capa oculta y una capa de salida con una funcion de activacion sigmoidal y una funcion de perdida de entropia binaria cruzada</h1>

In [25]:
inicio = time.time()
seed_value= 1234
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.5)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['recall'])

model.fit(X_train, y_train, epochs=100,
          batch_size=1000,
          verbose = 0)
modelos.append([model, 'ann'])
print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

Tiempo de ejecucion:  0.56 minutos


<h1>Evaluacion de los modelos con el set de entrenamiento según metricas exactitud, precision, recall y f1-score</h1>

In [27]:
for modelo in modelos:
    print('-----------  '+modelo[1]+'  -----------')
    inicio = time.time()
    y_hat = (modelo[0].predict(X_test)>=0.5).astype(int)
    print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')
    print('Exactitud:', accuracy_score(y_test,y_hat))
    print('Matriz de confusion:', confusion_matrix(y_test, y_hat))
    print('Reporte:\n', classification_report(y_test, y_hat))
    modelo.append((recall_score(y_test, y_hat) + precision_score(y_test, y_hat)) / 2)

-----------  lr  -----------
Tiempo de ejecucion:  0.00 minutos
Exactitud: 0.9837694275034429
Matriz de confusion: [[19747   327]
 [    3   255]]
Reporte:
               precision    recall  f1-score   support

         0.0       1.00      0.98      0.99     20074
         1.0       0.44      0.99      0.61       258

    accuracy                           0.98     20332
   macro avg       0.72      0.99      0.80     20332
weighted avg       0.99      0.98      0.99     20332

-----------  clf  -----------
Tiempo de ejecucion:  0.00 minutos
Exactitud: 0.9995573480228211
Matriz de confusion: [[20073     1]
 [    8   250]]
Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       1.00      0.97      0.98       258

    accuracy                           1.00     20332
   macro avg       1.00      0.98      0.99     20332
weighted avg       1.00      1.00      1.00     20332

-----------  knn  -----------


  File "C:\Users\ismae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\ismae\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ismae\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\ismae\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Tiempo de ejecucion:  0.16 minutos
Exactitud: 0.999262246704702
Matriz de confusion: [[20070     4]
 [   11   247]]
Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       0.98      0.96      0.97       258

    accuracy                           1.00     20332
   macro avg       0.99      0.98      0.99     20332
weighted avg       1.00      1.00      1.00     20332

-----------  svm  -----------
Tiempo de ejecucion:  0.00 minutos
Exactitud: 0.9989179618335629
Matriz de confusion: [[20068     6]
 [   16   242]]
Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       0.98      0.94      0.96       258

    accuracy                           1.00     20332
   macro avg       0.99      0.97      0.98     20332
weighted avg       1.00      1.00      1.00     20332

-----------  ann  -----------
636/636 ━━━━━━━━━━━━━━━━━━━━ 0s 5

<h1>Despliegue y entrenamiento extra del mejor modelo según promedio entre recall y precision</h1>

In [29]:
modelos=sorted(modelos, key=lambda x: x[-1], reverse=True)
modelos[0][0] = modelos[0][0].fit(X_test, y_test)
if (modelos[0][1] == 'ann'):
    modelos[0][0].save("modelo.keras")
else:
    joblib.dump(modelos[0][0], "modelo.pkl")